# Getting started with the Facebook Marketing API

## 1. Configure access to the Marketing API

Before querying data, we need to get some credentials to log in and access the Facebook Marketing API.

### Get your credentials

To start using the Facebook Marketing API, follow these steps:
1. log in and create a [Facebook Developer](https://developers.facebook.com) account
<img src="images/facebookDevelopers.png" width="600px"/>

2. go to *My Apps* and create an app (in the pop up, select *Manage Business Integrations*, then fill the form and create the app)
<img src="images/createApp.png" width="600px"/>

3. from the app's dashboard page, set up the Marketing API
<img src="images/marketingAPI.png" width="600px"/>

4. from the Marketing API menu, click on *Tools*, then select *ads_management* and *ads_read* as permissions and finally get the **Access Token**
<img src="images/accessToken.png" width="600px"/>

5. from the app's menu, click on *Settings > Basic* and get the **App ID** and the **App Secret**
<img src="images/appID.png" width="600px"/>

6. go to https://www.facebook.com/adsmanager and, from the URL, copy the part starting with *act=...* to get your **Ad account ID** (copy it in the form "act_1234...")
<img src="images/appID.png" width="600px"/>

To recap, make sure you have:
* an **app ID**
* an **app secret**
* an **access token**
* an **ad account id**

In [3]:
my_app_id = "your-app-id" #something like 12345...
my_app_secret = "your-app-secret" #something like cw4v44cr4...
my_access_token = "your-access-token" #something like Eafr5vGb...
my_account = "your-ad-account-id" #something like act_12345...

### Install the Facebook Business SDK for Python

To install the Facebook Business SDK for Python, follow the instructions in the Github page.

https://github.com/facebook/facebook-python-business-sdk

#### Test your installation

In [4]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount

app_id = my_app_id
app_secret = my_app_secret
access_token = my_access_token
FacebookAdsApi.init(app_id, app_secret, access_token)
account = AdAccount(my_account)
campaigns = account.get_campaigns()
print(campaigns)

[]


## 2. Query data

We are ready to query some data.

#### Import

In [93]:
import pandas as pd

from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount

#### Start connection with the API

These lines provide access to the Marketing API: remember to add them at the beginning of the notebook.

In [ ]:
app_id = my_app_id
app_secret = my_app_secret
access_token = my_access_token
FacebookAdsApi.init(app_id, app_secret, access_token)
account = AdAccount(my_account)

### A basic example

Let's start with a first basic example. Suppose we want to know how many 

> *people aged 20 to 50 living in Turin (Italy) with a college degree and interested in sports are on Facebook.*

To get an estimate, we can use the [Ads Manager](https://www.facebook.com/adsmanager/). From the main page:
1. click on *Create Ad*;
2. select *Reach* as campaign objective, i.e. show the advertisment to the maximum number of people. Other choices are available as well, depending on the research goals;
3. a screen shows other campaign features: do not modify them as we do not need to actually publish the advertisment. Click on *Next*;

At this point, we can select an audience by specifying targeting options, including **locations**, **demographics**, **interests**, **behaviors**, and **publisher platform**.

For our example, select:
1. **Location:** "Italy" and "People living in this Location"
<img src="images/location.png" width="600px"/>

2. **Demographics:** people aged between 20 and 50 years old
3. **Education level:** "College graduates"
4. **Interests:** "Sports"

<img src="images/demoAndInterest.png" width="600px"/>

[Note](https://www.facebook.com/business/help/440167386536513?id=176276233019487) that after selecting "College grad", we need to click on *Narrow Audience* and then select "Sports". In this case, the target are users who have a "college degree" **AND** are interested in "sports". By adding "Sports" at the same level of "College grad" (without clicking on *Narrow Audience*), we are instead targeting users who have a "college degree" **OR** are interested in "sports" and the estimated audience would be larger:
> The details you select aren't mutually inclusive. This means that if you add three interests (for example, movies, books and TV), we'll look for people who match the location, age, gender and language you selected, who also match with either "movies," or "books" or "TV."
If you want to narrow your intended audience more, select "Narrow Audience" after you add your initial choices. This will help us focus your audience makeup on people who must match at least one of your previously identified qualities and the additional details you add. For example, if you originally chose to include people who match with "movies," or "books" or "TV," and then you add "yoga," we'll now search for people who must match "yoga" and either "movies," "books" or "TV."

5. **Publishing platform (Placements):** "Facebook"

<img src="images/platform.png" width="600px"/>

As we can see on the upper right corner of the screen, the estimated audience is 140,000 users. Facebook [states](https://www.facebook.com/business/help/1665333080167380?id=176276233019487) that:
> This is an estimate of the size of the audience that's eligible to see your ad. It's based on your targeting criteria, ad placements and how many people were shown ads on Facebook apps and services in the past 30 days.
This is not an estimate of how many people will actually see your ad, and the number may change over time. It isn't designed to match population or census estimates.

### Retrieve the estimates in a programmatic way

Let's try to retrieve the estimates using the Marketing API. The following code snippet provides the solution, which we will examine in the following.

In [94]:
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount

# Access to the API
app_id = my_app_id
app_secret = my_app_secret
access_token = my_access_token
account = my_account
FacebookAdsApi.init(app_id, app_secret, access_token)
account = AdAccount(my_account)

# Specify targeting options
targeting_spec = {
    "geo_locations":{
        "cities":[
            {"key":"1192826"} #Turin (Italy)
        ],
        "location_types":["home"]
    },
    "age_min":20,
    "age_max":50,
    "education_statuses":[3], #College graduates
    "interests":[
        {
            "id":6003269553527,
            "name":"Sports"
        }
    ],
    "publisher_platforms":["facebook"]
}

params = {
    "optimization_goal":"REACH",
    "targeting_spec":targeting_spec
}

# Get user estimates
account.get_delivery_estimate(params=params)

[<AdAccountDeliveryEstimate> {
    "daily_outcomes_curve": [
        {
            "actions": 0,
            "impressions": 0,
            "reach": 0,
            "spend": 0
        }
    ],
    "estimate_dau": 117804,
    "estimate_mau": 140000,
    "estimate_ready": true
}]

Note that there are two types of estimate: the *estimate_dau*, i.e. *daily active users*, and the *estimate_mau*, i.e. *monthly active users*. I recommend using the *monthly active users* as they are more stable over time and the common choice for several research works in the literature.

We also note that "Turin" is specified with a numeric *key* that Facebook associates to it, as well as the education level and the interest. *How can we obtain such keys?* To do so, we need to use the [Targeting Search](https://developers.facebook.com/docs/marketing-api/audiences/reference/targeting-search) function. Let's see how it works with some examples.

In the folllowing, some of the more common targeting options are explained. For a complete and updated list of all the possible the targeting options available through the Marketing API, please refer to the [documentation](https://developers.facebook.com/docs/marketing-api/audiences/reference/basic-targeting).

#### Locations

Commonly used locations include: **country**, **regions**, **cities**, **zip codes**, and **custom locations**. While targeting other locations such as [neighborhoods](https://developers.facebook.com/docs/marketing-api/audiences/reference/targeting-search#geoarea) is also possible, they have been added recently and are not available for all the cities.

Also, it is possible to specify the *location_types* choosing one of:

* "home": People whose stated location in Facebook profile “current city” is in an area. Facebook validates this by IP and information from their friends’ profile locations.
* "recent": People whose recent location is selected area, as determined from mobile device data.
* "travel_in": People whose most recent location is selected area. Determined by mobile device data, and more than 100 miles from stated current city in their Facebook profile.

By default, the *location_type* is set to "home".

Remember that at least one location have to be specified in each query.

##### Countries

Let's try to target a country.

First, get Italy's country code.

In [7]:
from facebook_business.adobjects.targetingsearch import TargetingSearch

params = {
    "q": "italy",
    "type": "adgeolocation",
    "location_types": ["country"],
}

resp = TargetingSearch.search(params=params)
pd.DataFrame(resp)

,country_code,country_name,key,name,supports_city,supports_region,type
0,IT,Italy,IT,Italy,True,True,country


Second, use the country code as targeting specification.

In [10]:
# Specify targeting options
targeting_spec = {
    "geo_locations":{
        "countries":["IT"] #Italy
    }
}

params = {
    "optimization_goal":"REACH",
    "targeting_spec":targeting_spec
}

# Get user estimates
account.get_delivery_estimate(params=params)

[<AdAccountDeliveryEstimate> {
    "daily_outcomes_curve": [
        {
            "actions": 0,
            "impressions": 0,
            "reach": 0,
            "spend": 0
        }
    ],
    "estimate_dau": 28781279,
    "estimate_mau": 36000000,
    "estimate_ready": true
}]

##### Regions

The same applies for other location types.

In [95]:
from facebook_business.adobjects.targetingsearch import TargetingSearch

params = {
    "q": "piemonte",
    "type": "adgeolocation",
    "location_types": ["region"],
}

resp = TargetingSearch.search(params=params)
pd.DataFrame(resp)

,country_code,country_name,key,name,supports_city,supports_region,type
0,IT,Italy,1852,Piedmont,True,True,region


In [12]:
# Specify targeting options
targeting_spec = {
    "geo_locations":{
        "regions":[
            {"key":1852}  #Piedmont
        ]
    }
}

params = {
    "optimization_goal":"REACH",
    "targeting_spec":targeting_spec
}

# Get user estimates
account.get_delivery_estimate(params=params)

[<AdAccountDeliveryEstimate> {
    "daily_outcomes_curve": [
        {
            "actions": 0,
            "impressions": 0,
            "reach": 0,
            "spend": 0
        }
    ],
    "estimate_dau": 1906259,
    "estimate_mau": 2400000,
    "estimate_ready": true
}]

##### Cities

In [96]:
from facebook_business.adobjects.targetingsearch import TargetingSearch

params = {
    "q": "turin",
    "type": "adgeolocation",
    "location_types": ["city"],
}

resp = TargetingSearch.search(params=params)
pd.DataFrame(resp)

,country_code,country_name,geo_hierarchy_level,geo_hierarchy_name,key,name,region,region_id,supports_city,supports_region,type
0,IT,Italy,NaN,NaN,1192826,Turin,Piedmont,1852,True,True,city
1,US,United States,NaN,NaN,2435270,Turin,Georgia,3853,True,True,city
2,US,United States,NaN,NaN,2446332,Turin,Iowa,3858,True,True,city
3,CA,Canada,NaN,NaN,296931,Turin,Alberta,527,True,True,city
4,US,United States,NaN,NaN,2492035,Turin,New York,3875,True,True,city
5,HN,Honduras,SUBCITY,CITY,905411,Turin,Yoro Department,1590,True,True,subcity
6,HT,Haiti,SUBCITY,CITY,928316,Turin,Ouest Department,1618,True,True,subcity
7,BY,Belarus,SUBCITY,CITY,288885,Turin,Minsk Region,4455,True,True,subcity
8,CO,Colombia,SUBCITY,CITY,480961,Turin,Norte de Santander,734,True,True,subcity
9,CO,Colombia,NEIGHBORHOOD,NEIGHBORHOOD,2899758,Turin,Risaralda,737,True,True,neighborhood


In [15]:
# Specify targeting options
targeting_spec = {
    "geo_locations":{
        "cities":[
            {"key":1192826} #Turin (Italy)
        ]
    }
}

params = {
    "optimization_goal":"REACH",
    "targeting_spec":targeting_spec
}

# Get user estimates
account.get_delivery_estimate(params=params)

[<AdAccountDeliveryEstimate> {
    "daily_outcomes_curve": [
        {
            "actions": 0,
            "impressions": 0,
            "reach": 0,
            "spend": 0
        }
    ],
    "estimate_dau": 727354,
    "estimate_mau": 920000,
    "estimate_ready": true
}]

##### ZIP codes

ZIP codes are specified by placing the country code before the postal code, e.g. "[IT:10126](https://www.google.it/maps/place/10126+Torino+TO/@45.0424375,7.6577304,14z/data=!3m1!4b1!4m5!3m4!1s0x47886d4d2fb573e7:0x1c05e68093b7a400!8m2!3d45.0395418!4d7.6706443)".

In [16]:
# Specify targeting options
targeting_spec = {
    "geo_locations":{
        "zips":[
            {"key":"IT:10126"}
        ]
    }
}

params = {
    "optimization_goal":"REACH",
    "targeting_spec":targeting_spec
}

# Get user estimates
account.get_delivery_estimate(params=params)

[<AdAccountDeliveryEstimate> {
    "daily_outcomes_curve": [
        {
            "actions": 0,
            "impressions": 0,
            "reach": 0,
            "spend": 0
        }
    ],
    "estimate_dau": 31063,
    "estimate_mau": 37000,
    "estimate_ready": true
}]

##### Custom locations

Custom locations can be added in several ways. For example, we can specify a center (*latitude* and *longitude*) and an area around it (*radius*).

In [21]:
# Specify targeting options
targeting_spec = {
    "geo_locations":{
        "custom_locations":[
            {
                "latitude": 45.072606, 
                "longitude": 7.685585, 
                "radius": 5, 
                "distance_unit": "kilometer"
            }
        ]
    }
}

params = {
    "optimization_goal":"REACH",
    "targeting_spec":targeting_spec
}

# Get user estimates
account.get_delivery_estimate(params=params)

[<AdAccountDeliveryEstimate> {
    "daily_outcomes_curve": [
        {
            "actions": 0,
            "impressions": 0,
            "reach": 0,
            "spend": 0
        }
    ],
    "estimate_dau": 639689,
    "estimate_mau": 790000,
    "estimate_ready": true
}]

##### Multiple locations

We can even add multiple locations, also from different types.

In [29]:
# Specify targeting options
targeting_spec = {
    "geo_locations":{
        "cities":[
            {"key":1192826}, #Turin (Italy)
            {"key":1184376} #Milan (Italy)
        ],
        "countries":["JP"] #Japan
    }
}

params = {
    "optimization_goal":"REACH",
    "targeting_spec":targeting_spec
}

# Get user estimates
account.get_delivery_estimate(params=params)

[<AdAccountDeliveryEstimate> {
    "daily_outcomes_curve": [
        {
            "actions": 0,
            "impressions": 0,
            "reach": 0,
            "spend": 0
        }
    ],
    "estimate_dau": 28593599,
    "estimate_mau": 48000000,
    "estimate_ready": true
}]

#### Demographics

Basic demographics include **age** and **gender**. By default, the minimum age is set to 18, the maximum to 65+, and the gender to all (1=*male*, 2=*female*).

In [30]:
# Specify targeting options
targeting_spec = {
    "geo_locations":{
        "cities":[
            {"key":1192826}, #Turin (Italy)
        ]
    },
    "age_min":25,
    "age_max":45,
    "genders":[2] #female
}

params = {
    "optimization_goal":"REACH",
    "targeting_spec":targeting_spec
}

# Get user estimates
account.get_delivery_estimate(params=params)

[<AdAccountDeliveryEstimate> {
    "daily_outcomes_curve": [
        {
            "actions": 0,
            "impressions": 0,
            "reach": 0,
            "spend": 0
        }
    ],
    "estimate_dau": 170735,
    "estimate_mau": 200000,
    "estimate_ready": true
}]

#### Advanced targeting categories

Other advanced targeting characteristics can be selected, such as **education level**, **relationship status**, and **mobile device** used. A list of all the possible advanced categories is available in the [documentation](https://developers.facebook.com/docs/marketing-api/audiences/reference/advanced-targeting).

For example, all the possible education levels are available [here](https://developers.facebook.com/docs/marketing-api/audiences/reference/advanced-targeting#education_and_workplace).

In [67]:
# Specify targeting options
targeting_spec = {
    "geo_locations":{
        "cities":[
            {"key":1192826}, #Turin (Italy)
        ]
    },
    "education_statuses":[3] #College graduates
}

params = {
    "optimization_goal":"REACH",
    "targeting_spec":targeting_spec
}

# Get user estimates
account.get_delivery_estimate(params=params)

[<AdAccountDeliveryEstimate> {
    "daily_outcomes_curve": [
        {
            "actions": 0,
            "impressions": 0,
            "reach": 0,
            "spend": 0
        }
    ],
    "estimate_dau": 180913,
    "estimate_mau": 210000,
    "estimate_ready": true
}]

Or if we want to target users using a particular mobile device, we must specify the name of the device (*user_device*) and the OS (*user_os*).

In [68]:
from facebook_business.adobjects.targetingsearch import TargetingSearch

params = {
    "type": "adTargetingCategory",
    "class":"user_device"
}

resp = TargetingSearch.search(params=params)
pd.DataFrame(resp)

,audience_size,description,name,platform,type
0,88000000,iPads (all),iPad,iOS,user_device
1,620000000,iPhones (all),iPhone,iOS,user_device
2,1500000,iPods (all),iPod,iOS,user_device
3,3200000,Apple iPad,ipad,iOS,user_device
4,24000,Apple iPad 1,ipad 1,iOS,user_device
...,...,...,...,...,...
1114,3500000,Asus ZenFone Max Pro M1,zenfone max pro m1,Android,user_device
1115,270000,Asus ZenFone Selfie,zenfone selfie,Android,user_device
1116,370000,Zte Zmax Pro,zmax pro,Android,user_device
1117,150000,Lg Zone,zone,Android,user_device


In [80]:
# Specify targeting options
targeting_spec = {
    "geo_locations":{
        "cities":[
            {"key":1192826}, #Turin (Italy)
        ]
    },
    "user_device":["iPhone"], #iPhone users
    "user_os":["iOS"]
}

params = {
    "optimization_goal":"REACH",
    "targeting_spec":targeting_spec
}

# Get user estimates
account.get_delivery_estimate(params=params)

[<AdAccountDeliveryEstimate> {
    "daily_outcomes_curve": [
        {
            "actions": 0,
            "impressions": 0,
            "reach": 0,
            "spend": 0
        }
    ],
    "estimate_dau": 39690,
    "estimate_mau": 56000,
    "estimate_ready": true
}]

#### Interests

From the [documentation](https://developers.facebook.com/docs/marketing-api/audiences/reference/basic-targeting#interests):
> Target based on interests from someone's timeline, from Pages liked or from keywords associated with Pages or apps someone uses.

First, we need to retrieve the *id* associated to a particular interest. For example, suppose we want to target users interested in "Sports". The identifier associated to it is "6003269553527".

In [82]:
from facebook_business.adobjects.targetingsearch import TargetingSearch

params = {
    "q":"sports",
    "type":"adinterest"
}

resp = TargetingSearch.search(params=params)
pd.DataFrame(resp)

,audience_size,description,id,name,path,topic
0,1701633800,,6003269553527,Sports,"[Interests, Sports and outdoors, Sports]",Sports and outdoors
1,1728801040,,6008803895164,Sports and outdoors,"[Interests, Sports and outdoors]",NaN
2,197502252,,6003540150873,Sports games,"[Interests, Entertainment, Games, Sports games]",News and entertainment
3,171126510,None,6003545471303,Sportswear (activewear),"[Interests, Additional Interests, Sportswear (...",Sports and outdoors
4,145379430,None,6003493479631,Sports club,"[Interests, Additional Interests, Sports club]",Fitness and wellness
5,102292800,None,6003719306113,Champion (sportswear),"[Interests, Additional Interests, Champion (sp...",Business and industry
6,100316250,None,6003840055852,Sports car,"[Interests, Additional Interests, Sports car]",Hobbies and activities
7,79443980,None,6003070550582,Pool (cue sports),"[Interests, Additional Interests, Pool (cue sp...",Sports and outdoors


In [84]:
# Specify targeting options
targeting_spec = {
    "geo_locations":{
        "cities":[
            {"key":1192826}, #Turin (Italy)
        ]
    },
    "interests":[
        {
            "id":6003269553527,
            "name":"Sports" #the name is not needed
        }
    ]
}

params = {
    "optimization_goal":"REACH",
    "targeting_spec":targeting_spec
}

# Get user estimates
account.get_delivery_estimate(params=params)

[<AdAccountDeliveryEstimate> {
    "daily_outcomes_curve": [
        {
            "actions": 0,
            "impressions": 0,
            "reach": 0,
            "spend": 0
        }
    ],
    "estimate_dau": 677022,
    "estimate_mau": 760000,
    "estimate_ready": true
}]

#### Behaviors

From the [documentation](https://developers.facebook.com/docs/marketing-api/audiences/reference/basic-targeting#behaviors):
> Target based on digital activities, devices people use, past or intended purchases, and travel.

In the case of behaviors, we have to retrieve the list of all possible bahaviors, and then select one. For example, suppose we want to target all the users that are "Frequent travelers".

In [86]:
from facebook_business.adobjects.targetingsearch import TargetingSearch

params = {
    "type":"adTargetingCategory",
    "class":"behaviors"
}

resp = TargetingSearch.search(params=params)
behaviors_df = pd.DataFrame(resp)
behaviors_df.head()

,audience_size,description,id,name,path,real_time_cluster,type
0,1676488295,People whose activities on Facebook suggest th...,6002714895372,Frequent Travelers,"[Travel, Frequent Travelers]",False,behaviors
1,49496568,People who list themselves as small business o...,6002714898572,Small business owners,"[Digital activities, Small business owners]",False,behaviors
2,1047784,People who have used Facebook Payments in the ...,6002764392172,Facebook Payments users (90 days),"[Digital activities, Facebook Payments users (...",False,behaviors
3,99205746,People who are likely to adopt new technologie...,6003808923172,Technology early adopters,"[Digital activities, Technology early adopters]",False,behaviors
4,6909975,People who primarily access Facebook using Win...,6003986707172,Facebook access (OS): Windows 7,"[Digital activities, Operating System Used, Fa...",False,behaviors


In [87]:
behaviors_df[behaviors_df["name"].str.lower().str.contains("frequent travelers")]

,audience_size,description,id,name,path,real_time_cluster,type
0,1676488295,People whose activities on Facebook suggest th...,6002714895372,Frequent Travelers,"[Travel, Frequent Travelers]",False,behaviors


The *id* associated to "Frequent travelers" is "6002714895372" and we can then use it in the targeting specifications.

In [88]:
# Specify targeting options
targeting_spec = {
    "geo_locations":{
        "cities":[
            {"key":1192826}, #Turin (Italy)
        ]
    },
    "behaviors":[
        {
            "id":6002714895372,
            "name":"Frequent travelers" #the name is not needed
        }
    ]
}

params = {
    "optimization_goal":"REACH",
    "targeting_spec":targeting_spec
}

# Get user estimates
account.get_delivery_estimate(params=params)

[<AdAccountDeliveryEstimate> {
    "daily_outcomes_curve": [
        {
            "actions": 0,
            "impressions": 0,
            "reach": 0,
            "spend": 0
        }
    ],
    "estimate_dau": 408023,
    "estimate_mau": 480000,
    "estimate_ready": true
}]

#### Publisher platforms

Publisher platform can be choosen among "Facebook", "Instagram", "Audience Network", "Messenger". It is also possoble to choose the device used ("mobile" or "desktop"). More options are available [here](https://developers.facebook.com/docs/marketing-api/audiences/reference/placement-targeting#placement-targeting).

In [98]:
# Specify targeting options
targeting_spec = {
    "geo_locations":{
        "cities":[
            {"key":1192826}, #Turin (Italy)
        ]
    },
    "device_platforms":["mobile"],
    "publisher_platforms":["instagram"]
}

params = {
    "optimization_goal":"REACH",
    "targeting_spec":targeting_spec
}

# Get user estimates
account.get_delivery_estimate(params=params)

[<AdAccountDeliveryEstimate> {
    "daily_outcomes_curve": [
        {
            "actions": 0,
            "impressions": 0,
            "reach": 0,
            "spend": 0
        }
    ],
    "estimate_dau": 411203,
    "estimate_mau": 580000,
    "estimate_ready": true
}]

## 3. Limitations

* **Rate limit:** there is a limitation in the number of queries one can make in a given time range. When exceeding that limit, the access to the Marketing API may remain blocked for some minutes. To prevent blocking, we empirically determined a "safe" rate limit of 450 queries per hour, i.e. when making a large number of queries (more then ~100), make sure to add a delay of 8 seconds between queries.

* **1,000 threshold:** if the combination of targeting options for the query is too specific, the resulting audience estimate may be limited to a lower threshold of 1,000 users. To overcome this limitation, a recent method has been proposed [here](https://dl.acm.org/doi/abs/10.1145/3366423.3380118):
> To overcome this limitation, we propose an “exclusion” query, wherein in order to get an attribute-constrained population estimate of a small municipality S, it is first queried with another, larger, “reference” municipality R resulting in a combined query S+R. The difference between the combined query and the known reference municipality population ((S+R)-R) provides us an estimate for the small municipality S with a possible range of down to 100. This lower range is possible due to the finer resolution of results, which is not in 1,000s, but in the 100s.

## 4. References

* Main page of the [Facebook Marketing API](https://developers.facebook.com/docs/marketing-apis)
* [Facebook Business SDK in Python](https://github.com/facebook/facebook-python-business-sdk)
* [Ads Manager](https://www.facebook.com/adsmanager/)
* [Basic Targeting](https://developers.facebook.com/docs/marketing-api/audiences/reference/basic-targeting)
* [Targeting Search](https://developers.facebook.com/docs/marketing-api/audiences/reference/targeting-search)